In [1]:
import numpy as np
import os

from scipy.spatial import distance
import matplotlib.pyplot as plt    
import copy
from scipy import stats
from copy import deepcopy
import time
import re


In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

In [3]:
def extract_position(fname):
    neuron_data = np.loadtxt(fname)
    #print(fname)
    #print(neuron_data.shape)
    if neuron_data.ndim == 1:#  there exists files which do not have any neurites
        return np.zeros((1,3)), neuron_data[2:5]
    #else:
    soma = neuron_data[0,2:5] # considering only the positions/coordinates
    #print(soma)
    neurites = neuron_data[1:,2:5]
    return neurites,soma # neurites refers to either axon terminals/dendrites. It is a loose term. 


def count_connections(neuritesA, neuritesB, synaptic_cleft):
    # cdist is used to calculate euclidean distance between all endpoints of neuronA and all endpoints of neuronB
    # dist_axonTerminalsA_dendritesB is 2d array of dimension nrows(axonTerminalsA) x nrows(dendritesB)
    dist_neuritesA_neuritesB = distance.cdist(neuritesA, neuritesB) 
    #count all the dist less than 5um as a connection btw single neurite A and single neurite B
    connections = np.count_nonzero(dist_neuritesA_neuritesB <= synaptic_cleft) 
    return connections



def extract_adjacency(file_path, synaptic_cleft):    
    
    file_list=[f for f in sorted_alphanumeric(os.listdir(file_path)) if f.endswith('.swc')
                    and os.path.isfile(os.path.join(file_path, f))]
    
    neuron_names = []
    n_neurons = len(file_list)
    adjacency = np.zeros((n_neurons,n_neurons))
    
    
    start = time.time()
    counter = 0 
    
    for i in range(n_neurons):
        
        #print(i)
        j_list = list(range(i+1, n_neurons))

        for j in j_list:
            
            neuritesA, somaA = extract_position(file_path + file_list[i])
            neuritesB, somaB = extract_position(file_path + file_list[j])
           
            if np.all(neuritesA==0) or np.all(neuritesB==0):#np.zeros((1,3)):
                adjacency[i,j] = 0
                adjacency[j,i] = 0
            else:
                num_connections = count_connections(neuritesA, neuritesB, synaptic_cleft) 
                adjacency[i,j] = num_connections
                adjacency[j,i] = num_connections
        counter+=1
        if (counter%100)==0:
            end=time.time()
            print(i, "%.3f min"%((end-start)/60))
            start = time.time()
            
        neuron_names.append(file_list[i][:-4])# [:-4] becoz we don't want ".swc" in the end
    
    return adjacency, neuron_names 


In [ ]:
# Code to obtain undirected network or adjacency matrices for proximity range between 1 - 10 um.

#def run_parallel(synaptic_cleft):
#    file_path = "./strhl_num_1_dataset/"
#    adjacency, neuron_names = extract_adjacency(file_path, synaptic_cleft=synaptic_cleft)
#    np.save("undir_adjacency_synaptic_cleft_%.3f um.npy"%synaptic_cleft, adjacency.astype(np.int64))

#import multiprocessing
#pool_obj = multiprocessing.Pool(processes=32)

#_ = pool_obj.map(run_parallel, np.linspace(1,10,32))


In [4]:
# Here we use proximity range and synaptic cleft interchangeably

# for proximity range or synaptic cleft equal to 5um

synaptic_cleft = 5

file_path = "./strhl_num_1_dataset/"
adjacency, neuron_names = extract_adjacency(file_path, synaptic_cleft=synaptic_cleft)


99 16.675 min
199 26.583 min
299 11.742 min
399 8.120 min
499 8.008 min
599 9.084 min
699 9.359 min
799 9.294 min
899 8.593 min
999 9.233 min
1099 8.543 min
1199 8.322 min
1299 7.261 min
1399 9.743 min
1499 6.365 min
1599 5.962 min
1699 10.266 min
1799 8.241 min
1899 6.679 min
1999 5.315 min
2099 5.025 min
2199 4.391 min
2299 3.734 min
2399 3.426 min
2499 3.069 min
2599 3.306 min
2699 2.879 min
2799 2.910 min
2899 1.932 min
2999 1.198 min
3099 0.720 min


In [5]:
np.save("undirected_adj_synaptic_cleft_%.3f um.npy"%synaptic_cleft, adjacency.astype(np.int64))
